In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
#import torch.cuda
import random
from itertools import chain as chain
from torch.distributions.multivariate_normal import MultivariateNormal

#conda activate base
cudaOn = True

In [2]:
#tepLoc = "C:/Users/Charlie/Desktop/TEP_Data/"
#tepTrain = tepLoc + "TEP_Faulty_Training.csv"
#tepTest = tepLoc + "TEP_Faulty_Testing.csv"
tepTrain = '/content/TEP_Faulty_Training.csv'
tepTest = '/content/TEP_Faulty_Testing.csv'

scaler = StandardScaler()

#data = pd.read_csv('c172_file_1.csv')
data = pd.read_csv(tepTrain)
dataTest = pd.read_csv(tepTest)
#I should only be scaling the values that are double values, scaling fault number and other stuff just kinda fucks output
#scaler.fit(dataTest)
#data = pd.DataFrame(scaler.transform(data))
#dataTest = pd.DataFrame(scaler.transform(dataTest))


In [3]:
data = data.drop('Unnamed: 0',axis=1)
data = data.drop('faultNumber',axis=1)
data = data.drop('simulationRun',axis=1)
data = data.drop('sample',axis=1)
faultNumbersTest = dataTest.get('faultNumber')
dataTest = dataTest.drop('Unnamed: 0',axis=1)
dataTest = dataTest.drop('faultNumber',axis=1)
dataTest = dataTest.drop('simulationRun',axis=1)
dataTest = dataTest.drop('sample',axis=1)
dataTest = dataTest.iloc(0)[0:18849]



In [4]:
print(data)

         xmeas_1  xmeas_2  xmeas_3  xmeas_4  ...   xmv_8   xmv_9  xmv_10  xmv_11
0        0.25038   3674.0   4529.0   9.2320  ...  47.757  47.510  41.258  18.447
1        0.25109   3659.4   4556.6   9.4264  ...  43.692  47.427  41.359  17.194
2        0.25038   3660.3   4477.8   9.4426  ...  46.699  47.468  41.199  20.530
3        0.24977   3661.3   4512.1   9.4776  ...  47.541  47.658  41.643  18.089
4        0.29405   3679.0   4497.0   9.3381  ...  47.645  47.346  41.507  18.461
...          ...      ...      ...      ...  ...     ...     ...     ...     ...
3060044  0.66483   3684.3   4529.0   9.0643  ...  44.727  45.511  41.115  15.773
3060045  0.66528   3704.7   4571.7   9.0696  ...  44.257  46.020  41.014  20.038
3060046  0.66975   3670.9   4509.9   8.9267  ...  42.749  46.182  40.802  18.633
3060047  0.66847   3650.8   4507.2   8.8937  ...  45.676  46.684  41.316  19.655
3060048  0.74968   3674.4   4523.2   8.9405  ...     NaN     NaN     NaN     NaN

[3060049 rows x 52 columns]

In [4]:
print(dataTest)
numVariables = 52

       xmeas_1  xmeas_2  xmeas_3  xmeas_4  ...   xmv_8   xmv_9  xmv_10  xmv_11
0      0.25171   3672.4   4466.3   9.5122  ...  47.955  47.300  42.100  15.345
1      0.25234   3642.2   4568.7   9.4145  ...  45.038  47.502  40.553  16.063
2      0.24840   3643.1   4507.5   9.2901  ...  44.553  47.479  41.341  20.452
3      0.25153   3628.3   4519.3   9.3347  ...  48.048  47.440  40.780  17.123
4      0.21763   3655.8   4571.0   9.3087  ...  44.678  47.530  41.089  18.681
...        ...      ...      ...      ...  ...     ...     ...     ...     ...
18844  0.23512   3659.0   4491.4   9.4481  ...  45.047  48.360  40.931  20.159
18845  0.23559   3666.5   4443.3   9.4082  ...  43.727  49.054  41.958  18.813
18846  0.27699   3676.6   4493.5   9.4221  ...  48.630  49.507  41.300  18.780
18847  0.27973   3701.7   4464.5   9.3144  ...  45.989  49.697  41.432  15.319
18848  0.29423   3707.4   4553.7   9.3818  ...  45.877  49.894  41.474  16.663

[18849 rows x 52 columns]


In [8]:
def tep_testing_stepped(dat,step_size):
    res = []
    ind = 0
    scale = MinMaxScaler().fit(dat)
    dat = scale.transform(dat)
    #print(int((len(data)/step_size)))
    for i in range(int((len(dat)/step_size))):
        if ind + step_size < len(dat):
            step = []
            for j in range(step_size):
              #print(data.iloc(0)[ind])#[ind])
              step.append(data.iloc(0)[ind])
              ind = ind + 1
            res.append(step)
    return res

t = tep_testing_stepped(dataTest,5)
#t = to_var(t)
#print(len(t[0][0]))

In [9]:
#Split and reshape the data set by step_size , use min-max or stanrdardlize method to rescale the data
def Splitting_dataset(data, step_size, scale=True, scaler_type=MinMaxScaler):
        l = len(data) 
        data = scaler_type().fit_transform(data)
        Xs = []
        Ys = []
        for i in range(0, (len(data) - step_size)):
            Xs.append(data[i:i+step_size])
            Ys.append(data[i:i+step_size])
        train_x, test_x, train_y, test_y = [np.array(x) for x in train_test_split(Xs, Ys)]
        assert train_x.shape[2] == test_x.shape[2] == (data.shape[1] if (type(data) == np.ndarray) else len(data))
        return  (train_x.shape[2], train_x, train_y, test_x, test_y)
    
def get_batch(x, batch_size):
    """Made with taking test_x or XX as input"""
    t = 0
    while t >= 0:
        x_mod = len(x) % batch_size
        start = random.random() * (len(x)-x_mod)
        start = int(start)
        if start + batch_size < len(x):
            t = t-1
    batch = x[start:(start+batch_size)]
    #print(batch.shape)
    return batch

def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def loss_fn(recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    
        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu**2 -  logvar.exp())
        return BCE + KLD
    


In [10]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=27, z_dim=31, n_flow_steps=1):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, h_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(h_dim, z_dim*2) #is it saying its getting a mu and a var for each z dim out?
            
            #how can I represent the encoder as a distribution acting as the prior?
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, image_size),
            nn.Sigmoid()
        )
    
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_() 
        esp = to_var(torch.randn(*mu.size()))
        z = mu + std * esp
        return z
    
    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = torch.chunk(h, 2, dim=1)
        #print(mu.shape)
        #print(logvar.shape)
        z = self.reparameterize(mu, logvar)
        #print(z.shape)
        #z = z.float()
        z = model(z)
        #print(z)
        tensorZ = z[0]#torch.tensor(z[0])
        #print(tensorZ.shape)
        #print(z[0])
        return self.decoder(tensorZ), mu, logvar
    

class stacked_NVP(nn.Module):
    def __init__(self, d, k, hidden, n):
        super().__init__()
        self.bijectors = nn.ModuleList([
            R_NVP(d, k, hidden=hidden) for _ in range(n)
        ])
        self.flips = [True if i%2 else False for i in range(n)]
        
    def forward(self, x):
        log_jacobs = []

        for bijector, f in zip(self.bijectors, self.flips):
            x, log_pz, lj = bijector(x, flip=f)
            log_jacobs.append(lj)
        
        return x, log_pz, sum(log_jacobs)
    
    def inverse(self, z):
        for bijector, f in zip(reversed(self.bijectors), reversed(self.flips)):
            z = bijector.inverse(z, flip=f)
        return z
    
class R_NVP(nn.Module):
    def __init__(self, d, k, hidden):
        super().__init__()
        self.d, self.k = d, k
        self.sig_net = nn.Sequential(
                    nn.Linear(k, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, d - k))

        self.mu_net = nn.Sequential(
                    nn.Linear(k, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, d - k))

    def forward(self, x, flip=False):
        x1, x2 = x[:, :self.k], x[:, self.k:] 

        if flip:
            x2, x1 = x1, x2
        
        # forward
        sig = self.sig_net(x1)
        z1, z2 = x1, x2 * torch.exp(sig) + self.mu_net(x1)
        
        if flip:
            z2, z1 = z1, z2
        
        z_hat = torch.cat([z1, z2], dim=-1)

        log_pz = base_dist.log_prob(z_hat)
        log_jacob = sig.sum(-1)
        
        return z_hat, log_pz, log_jacob
    
    def inverse(self, Z, flip=False):
        z1, z2 = Z[:, :self.k], Z[:, self.k:] 
        
        if flip:
            z2, z1 = z1, z2
        
        x1 = z1
        x2 = (z2 - self.mu_net(z1)) * torch.exp(-self.sig_net(z1))
        
        if flip:
            x2, x1 = x1, x2
        return torch.cat([x1, x2], -1)

In [ ]:
step_size = 5
batch = 512
index_step_length = numVariables
epochs = 100

d = 2
k = 1

base_mu, base_cov = torch.zeros(2), torch.eye(2)

base_mu = to_var(base_mu.double())
base_cov = to_var(base_cov.double())

base_dist = MultivariateNormal(base_mu, base_cov)
#---------------------------------------------------------------------------------------------------------------------------------
labels, X, Y, XX, YY = Splitting_dataset(data, step_size)
#XX.cuda()
demo = VAE(index_step_length,h_dim=7,z_dim=2)
model = stacked_NVP(d, k, hidden=512,n=5)
demo.double()
model.double()
    
#next set of tests should be with n=3, last set was with n=1
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3)
optimizer2 = torch.optim.RMSprop(demo.parameters(), lr=1e-3)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.999)

if torch.cuda.is_available() & cudaOn:
    demo.cuda()
    print("demo done")
    model.cuda()
    print("model done")

In [ ]:
idx = 0

anomaly_history = []
loss_history = []
avgSum = 0
avgCount = 0

for epoch in range(epochs):
    b = get_batch(X,batch)
    #print(range(batch))
    for i in range(batch):
        #localX = torch.tensor(b[i].cuda())
        localX = to_var(torch.tensor(b[i]))
        recon, mu, logvar = demo(localX)
        loss = loss_fn(recon, localX, mu, logvar) #doing kl-divergence loss correctly
        """This bound (kl loss) provides a unified objective function for 
        op-timization of both the parameters θ and φ of the model and variational approximation, respectively."""
        optimizer.zero_grad()
        optimizer2.zero_grad()
        loss.backward()
        optimizer.step()
        optimizer2.step()
        scheduler.step()
        idx = idx + 1

        avgSum = avgSum + torch.mean(loss/batch)
        avgCount = avgCount + 1
        anomaly_score = torch.mean(localX/recon)

        if idx%30 == 0:
            loss_history.append(avgSum/avgCount)
            anomaly_history.append(anomaly_score)
            avgSum = 0
            avgCount = 0

        if idx%100 == 0:
            print("Epoch[{}/{}] Loss: {:.3f}".format(epoch+1, epochs, loss.data.item()/batch))
            
p1 = plt.figure()
plt.plot(loss_history,'g-',label='h 10,z 2')


step_start = 0
anomalies = []
county = 0
#print(type(XX))
#print(len(XX))
#print(XX.shape)
for step in t:
  step = to_var(torch.tensor(step))
  if True:
      #step = torch.tensor(XX[step_start:step_start+step_size])[0]
      recon,_,_ = demo(step)
      anom = torch.mean(step/recon)
      if county%100 == 0:
        print("step: " + str(step))
        print("recon: " + str(recon))
        print(anom)
      anomalies.append(anom)
      step_start = step_start + 1
      county = county + 1
        
p3 = plt.figure()
plt.plot(anomalies, 'g-')